# Part 1: Blast Messanger

### Setup

In [2]:
from langchain_community.llms import Ollama
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, PromptTemplate
from langchain.memory import ChatMessageHistory
from operator import itemgetter

from dotenv import load_dotenv
import os

# Load environmental variables
load_dotenv()

True

In [3]:
ollama = Ollama(
        base_url="http://10.30.16.100:11434",
        model="llama2")

To use, fill in the example scenario with details for the situation you want to cover.

In [4]:
scenario_template = ChatPromptTemplate.from_template("""
Situation: {situation}
Urgency: {urgency}
Current Time: {current_time}
Alternative Routes: {alternative_routes}
Support: {support_contact}
Safety Precautions: {safety_precautions}
Compensation: {compensation}

Response:
""")

def generate_communication(scenario_details):
    response = lang_chain.run(scenario_template, scenario_details)
    return response

# Insert your scenario here
scenario_details = {
    'situation': 'A tree has fallen on the tracks between 30th Street and Suburban Station.',
    'urgency': 'High',
    'current_time': '3:45 PM',
    'alternative_routes': 'Use Route 21 Bus instead',
    'support_contact': 'Contact Transit Support at 123-456-7890',
    'safety_precautions': 'Avoid the area near the incident. Follow police directions.',
    'compensation': 'Affected tickets will be refunded.'
}



### Define the prompt templates 

In [11]:
# Define the prompt templates
web_prompt = ChatPromptTemplate.from_template("""
  Generate a website statement for the following transportation incident. Use this as an example: Dear valued Amtrak passengers,

We regret to inform you that due to a tragic incident at the Trenton Station, the Amtrak service from Philadelphia to New York Penn Station has been temporarily suspended as of 4:06 pm today. We understand the urgency and inconvenience this may cause our passengers, and we are working diligently to provide alternative travel options.

For those who need to reach New York Penn Station, we recommend using the NJ Transit Northeast Corridor Line, specifically train NEC 3896, which is currently operating on schedule. We apologize for any inconvenience this may cause and appreciate your understanding during this difficult time.

If you require further assistance or have any questions, please don't hesitate to contact our customer support team at 222-333-4444 or use our chat bot for immediate help. We are here to support you in any way we can.

As a gesture of our appreciation for your patience and understanding, all tickets for the affected Amtrak service will be fully refunded. We value your loyalty and are committed to ensuring that you reach your destination safely and comfortably.

Thank you for your cooperation during this challenging situation. Our thoughts are with those affected by this unfortunate event.

Sincerely,
The Amtrak Team


Situation: {situation}
Urgency: {urgency}
Current Time: {current_time}
Alternative Routes: {alternative_routes}
Support: {support_contact}
Safety Precautions: {safety_precautions}
Compensation: {compensation}

Response:
"""
)

tweet_prompt = ChatPromptTemplate.from_template("""
  Generate a tweet that does not use '#' for the following transportation incident: Amtrak Alert: Service to NY Penn from Philadelphia suspended due to an incident at Trenton Station at 4:06 PM. Take NJ Transit NEC 3896 as an alternative. Call 222-333-4444 or use our chat bot for support. All tickets will be refunded. We apologize for any inconvenience caused. 

Situation: {situation}
Urgency: {urgency}
Current Time: {current_time}
Alternative Routes: {alternative_routes}
Support: {support_contact}
Safety Precautions: {safety_precautions}
Compensation: {compensation}

Response:
"""
)

blast_prompt = ChatPromptTemplate.from_template("""
  Generate a blast message for the following transportation incident:

Situation: {situation}
Urgency: {urgency}
Current Time: {current_time}
Alternative Routes: {alternative_routes}
Support: {support_contact}
Safety Precautions: {safety_precautions}
Compensation: {compensation}

Response:
"""
)


### Usage 1: Creating a web statement

In [10]:
scenario_chain3 = (web_prompt|ollama)
scenario_chain3.invoke(scenario_details)

'Dear valued passengers,\n\nWe are writing to inform you of a transportation incident that may affect your travel plans. Due to a tree falling on the tracks between 30th Street and Suburban Station, the Amtrak service from Philadelphia to New York Penn Station has been temporarily suspended as of 3:45 PM today.\n\nWe understand the urgency and inconvenience this may cause, and we are working diligently to provide alternative travel options. As a result, we recommend using the Route 21 Bus instead of the Amtrak service. Please note that this route may take longer than usual, and we apologize for any inconvenience this may cause.\n\nFor those who need assistance or have questions, please contact our Transit Support team at 123-456-7890. We are here to support you in any way we can.\n\nIn addition, as a gesture of goodwill, all tickets for the affected Amtrak service will be fully refunded. We value your loyalty and appreciate your patience and understanding during this challenging situat

### Usage 2: Creating a tweet

In [12]:
scenario_chain = (tweet_prompt|ollama)
scenario_chain.invoke(scenario_details)

'"BREAKING: Amtrak service between Philly and NYC suspended due to tree on tracks at Trenton Station. Use NJ Transit NEC 3896 as alternative. Call 222-333-4444 or chat with us for support. All tickets will be refunded. Avoid area near incident and follow police directions. #AmtrakAlert #NJTransit #PublicSafety"'

### Usage 3: Traditional blast message/text

In [13]:
scenario_chain2 = (blast_prompt|ollama)
scenario_chain2.invoke(scenario_details)

'BLAST MESSAGE - TREE ON TRACKS BETWEEN 30TH STREET AND SUBURBAN STATION\n\nURGENCY LEVEL: HIGH\n\n current time: 3:45 PM\n\nDUE TO A TREE FALLING ON THE TRACKS BETWEEN 30TH STREET AND SUBURBAN STATION, THERE IS A DELAY IN SERVICE. PLEASE USE ALTERNATIVE ROUTES SUCH AS THE ROUTE 21 BUS.\n\nFOR SUPPORT, CONTACT TRANSIT SUPPORT AT 123-456-7890.\n\nPLEASE FOLLOW POLICE DIRECTIONS FOR SAFETY PRECAUTIONS IN THE AREA NEAR THE INCIDENT.\n\nAFFECTED TICKETS WILL BE REFUNDED.\n\nTHANK YOU FOR YOUR UNDERSTANDING AND COOPERATION.'

# Part 2: ChatBot

### Setup

In [16]:
import openai
import requests
import json
from openai import OpenAI
from dotenv import load_dotenv

from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.llms import Ollama

from langchain_anthropic import ChatAnthropic

import pandas as pd

__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [17]:
_ = load_dotenv()

### Setting up the API Key

In [18]:
client = OpenAI(
    base_url = 'http://10.30.16.100:11434/v1',
    api_key='ollama', # required, but unused
)
opus = ChatAnthropic(model='claude-3-opus-20240229')

In [24]:
loader = CSVLoader(file_path="situations.csv")

docs = loader.load()

In [25]:
embeddings = OllamaEmbeddings(
    base_url="http://10.30.16.100:11434",
    model="nomic-embed-text")

vectorstore = Chroma.from_documents(documents=docs, embedding=embeddings)

In [26]:
retriever = vectorstore.as_retriever(search_type="similarity", 
                                     search_kwargs={"k": 1})



### Create our Own RAG Prompt for the Chatbot

In [56]:
our_prompt3 = '''
You are a public relations chatbot that specializes in helping riders find the specificities of their 
alternative route and solve their problem as well as calm their nerves. 

You have access to up to date information. Only use this information if it is time relevant (i.e. not in the future). Make your answer based on the current time right now. If the time is not provided, please ask the user what time it is. 
If you can't find relevant information to help the customer do not give a specific answer just try and
reassure them that more information will be available soon. If they are distressed, addressing their feelings be a support system for them to feel better. Otherwise, focus on addressing their question.

Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"
'''


In [57]:
prompt = hub.pull("rlm/rag-prompt")
prompt.messages[0].prompt.template = our_prompt3

In [58]:


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_result = (retriever | format_docs)

norag_chain = (
    prompt
    | opus
    | StrOutputParser()
)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | opus
    | StrOutputParser()
)

There are three main uses for this chatbot. To use it, replace the queries with your own situation.

### 1. Providing updates on situations. 


In [65]:
query = "The time is 8:31am. Why is my SEPTA late?"

In [66]:
rag_chain.invoke({"input": query})

'I understand your frustration with the delay. Route 10 is currently suspended between 63rd-Malvern and 13th-Market due to an urgent situation. As an alternative, you can use SEPTA bus routes 7, 38, or 42, which provide service along parts of the Route 10 corridor.'

### 2. Giving alternative routes/directions

In [61]:
query2 = "The time is 2:41 pm. How can I get to Maryland?"

In [62]:
rag_chain.invoke({"input": query2})

'I understand you want to get to Maryland, but Amtrak service at the New Carrollton Station is currently suspended due to safety concerns. Your safety is our top priority, so I recommend taking an alternative route, such as the MARC train from Washington Union Station to Baltimore Penn Station, then transferring to the SEPTA Regional Rail to continue to Philadelphia. Please check the MARC and SEPTA schedules for the most up-to-date information.'

### 3. Providing emotional support

In [63]:
query3 = "I'm so stressed out right now."

In [64]:
rag_chain.invoke({"input": query3})

"I understand this delay is frustrating, but please know we're doing everything possible to resume normal service quickly. In the meantime, you can take the 4 or 5 line as an alternative, which runs parallel to the 6. I'm here to help if you need any other information."